<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C2-Writing_A_Sniffer/Host_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Decoding ICMP__*

Now that we can fully decode the IP layer of any sniffed packets, we have to be able to decode the __ICMP__ responses that our scanner will elicit from sending __UDP__ datagrams to closed ports. ICMP messages can vary greatly in their contents, but each message contains three elements that stay consistent: The type, code, and checksum fields. The type and code fields tell the receiving host what type of ICMP message is arriving, which then dictates how to decode it properly.

For the purpose of our scanner, we are looking for a type value of 3 and a code value of 3. This corresponds to the __Destination Unreachable__ class of ICMP messages, and the code value of 3 indicates that the __Port Unreachable__ error has been caused. Refer to __Figure 3-3__ for a diagram of a Destination Unreachable ICMP message.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C2-Writing_A_Sniffer/ICMP_Destination_Unreachable_Message.png?raw=true" alt="From Client to Server" width="50%">
<p style="text-align:center"><em><strong>Figure 3-3:</strong> Diagram of Destination Unreachable ICMP message</em></p>
</div>

As you can see, the first 8 bits are the type, and the second 8 bits contain our ICMP code. One interesting thing to note is that when a host sends one of these ICMP messages, it actually includes the IP header of the originating message that generated the response. We can also see that we will double-check against 8 bytes of the original datagram that was sent in order to make sure our scanner generated the ICMP response. To do so, we simply slice off the last 8 bytes of the received buffer to pull out the magic string that our scanner sends.

Let's add some more code to our previous sniffer to inlcude the ability to decode ICMP packets. Let's save our previous file as __sniffer_with_icmp.py__ and add the following code:

In [ ]:
import ipaddress
import os
import socket
import struct
import sys

class IP:
    def __init__(self, buff=None):
        header = struct.unpack('<' + "BBHHHBBH4s4s", buff)
        self.ver = header[0] >> 4
        self.ihl = header[0] & 0xF

        self.tos = header[1]
        self.len = header[2]
        self.id = header[3]
        self.offset = header[4]
        self.ttl = header[5]
        self.protocol_num = header[6]
        self.sum = header[7]
        self.src = header[8]
        self.dst = header[9]

        self.src_address = ipaddress.ip_address(self.src)
        self.dst_address = ipaddress.ip_address(self.dst)

        self.protocol_map = {1: "ICMP", 6: "TCP", 17: "UDP"}
        try:
            self.protocol = self.protocol_map[self.protocol_num]
        except Exception as e:
            print("%s No protocol for %s" % (e, self.protocol_num))
            self.protocol = str(self.protocol_num)

class ICMP: #[1]
    def __init__(self, buff):
        header = struct.unpack("<" + "BBHHH", buff)
        self.type = header[0]
        self.code = header[1]
        self.sum = header[2]
        self.id = header[3]
        self.seq = header[4]

def sniff(host):
    if os.name == "nt":
        socket_protocol = socket.IPPROTO_IP
    else:
        socket_protocol = socket.IPPROTO_ICMP

    sniffer = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket_protocol)
    sniffer.bind((host, 0))
    sniffer.setsockopt(socket.IPPROTO_IP, socket.IP_HDRINCL, 1)

    if os.name == "nt":
        sniffer.ioctl(socket.STO_RCVALL, socket.RCVALL_ON)

    try:
        while True:
            raw_buffer = sniffer.recvfrom(65535)[0]
            ip_header = IP(raw_buffer[0:20])
            # If it's ICMP, we want it
            if ip_header.protocol == "ICMP": #[2]
                print("Protocol: %s %s -> %s" % (ip_header.protocol,
                                                 ip_header.src_address,
                                                 ip_header.dst_address))
                print(f"Version: {ip_header.ver}")
                print(f"Header Length: {ip_header.ihl} TTL: {ip_header.ttl}")

                # Calculate where our ICMP packet starts
                offset = ip_header.ihl * 4 #[3]
                buff = raw_buffer[offset:offset + 8]
                # Create our ICMP structure
                icmp_header = ICMP(buff) #[4]
                print("ICMP -> Type: %s Code: %s\n" % 
                      (icmp_header.type, icmp_header.code))
                
    except KeyboardInterrupt:
        if os.name == "nt":
            sniffer.ioctl(socket.SIO_RCVALL, socket.RCVALL_OFF)
        sys.exit()

if __name__ == "__main__":
    if len(sys.argv) == 2:
        host = sys.argv[1]
    else:
        host = "192.168.1.203"
    sniff(host)

This simple piece of code creates an ICMP structure __[1]__ underneath our existing IP structure. When the main packet-receiving loop determines that we have received an ICMP packet __[2]__, we calculate the offset in the raw packet where the ICMP body lives __[3]__ and then create our buffer __[4]__ and print out the __type__ and __code__ fields. The length calculation is based on the IP header __ihl__ field, which indicates the number of 32-bit words (4-byte chunks) contained in the IP header. So by multiplying this field by 4, we know the size of the IP header and thus when the next network layer (ICMP in this case) begins.

If we quickly run this code with our typical ping test, our output should now be slightly different:
```
Protocol: ICMP 74.125.226.78 -> 192.168.0.190
ICMP -> Type: 0 Code: 0
```
This indicates that the ping (__ICMP Echo__) responses are being correctly received and decoded. We are now ready to implement the last bit of logic to send out the UDP datagrams and to interpret their results.

Now let's add the use of the __ipaddress__ module so that we can cover an entire subnet with our host discovery scan. Save your __sniffer_with_icmp.py__ script as __scanner.py__ and add the following code:

In [ ]:
import ipaddress
import os
import socket
import struct
import sys
import threading
import time

# Subnet to target
SUBNET = "192.168.1.0/24"
# Magic string we'll check ICMP responses for
MESSAGE = "DIOBOIA" #[1]

class IP:
    def __init__(self, buff=None):
        header = struct.unpack('<' + "BBHHHBBH4s4s", buff)
        self.ver = header[0] >> 4
        self.ihl = header[0] & 0xF

        self.tos = header[1]
        self.len = header[2]
        self.id = header[3]
        self.offset = header[4]
        self.ttl = header[5]
        self.protocol_num = header[6]
        self.sum = header[7]
        self.src = header[8]
        self.dst = header[9]

        self.src_address = ipaddress.ip_address(self.src)
        self.dst_address = ipaddress.ip_address(self.dst)

        self.protocol_map = {1: "ICMP", 6: "TCP", 17: "UDP"}
        try:
            self.protocol = self.protocol_map[self.protocol_num]
        except Exception as e:
            print("%s No protocol for %s" % (e, self.protocol_num))
            self.protocol = str(self.protocol_num)

class ICMP:
    def __init__(self, buff):
        header = struct.unpack("<" + "BBHHH", buff)
        self.type = header[0]
        self.code = header[1]
        self.sum = header[2]
        self.id = header[3]
        self.seq = header[4]

# This sprays out UDP datagrams with our magic message
def udp_sender(): #[2]
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sender:
        for ip in ipaddress.ip_network(SUBNET).hosts():
            sender.sendto(bytes(MESSAGE, "utf-8"), (str(ip), 65212))

class Scanner: #[3]
    def __init__(self, host):
        self.host = host
        if os.name == "nt":
            socket_protocol = socket.IPPROTO_IP
        else:
            socket_protocol = socket.IPPROTO_ICMP
        
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_RAW, 
                                    socket_protocol)
        self.socket.bind((host, 0))

        self.socket.setsockopt(socket.IPPROTO_IP, socket.IP_HDRINCL, 1)

        if os.name == "nt":
            self.socket.ioctl(socket.SIO_RCVALL, socket.RCVALL_ON)

    def sniff(self): #[4]
        hosts_up = set([f"{str(self.host)} *"])
        try:
            while True:
                # Read a packet
                raw_buffer = self.socket.recvfrom(65535)[0]
                # Create an IP header from the first 20 bytes
                ip_header = IP(raw_buffer[0:20])
                # If it's ICMP, we want it
                if ip_header.protocol == "ICMP":
                    offset = ip_header.ihl * 4
                    buff = raw_buffer[offset:offset + 8]
                    icmp_header = ICMP(buff)
                    # Check for CODE 3 and TYPE 3
                    if icmp_header.code == 3 and icmp_header.type == 3:
                        if ipaddress.ip_address(ip_header.src_address) in #[5]
                            ipaddress.IPv4Network(SUBNET):
                            # Make sure it has our magic message
                            if raw_buffer[len(raw_buffer) - len(MESSAGE):] == #[6]
                                bytes(MESSAGE, "utf-8"):
                                tgt = str(ip_header.src_address)
                                if tgt != self.host and tgt not in hosts_up:
                                    hosts_up.add(str(ip_header.src_address))
                                    print(f"Host Up: {tgt}") #[7]
        # Handle CTRL-C
        except KeyboardInterrupt: #[8]
            if os.name == "nt":
                self.socket.ioctl(socket.SIO_RCVALL, socket.RCVALL_OFF)

            print("\nUser interrupted.")
            if hosts_up:
                print(f"\n\nSummary: Hosts up on {SUBNET}")
            for host in sorted(hosts_up):
                print(f"{host}")
            print('')
            sys.exit()

if __name__ == "__main__":
    if len(sys.argv) == 2:
        host = sys.argv[1]
    else:
        host = "192.168.1.203"
    s = Scanner(host)
    time.sleep(5)
    t = threading.Thread(target=udp_sender) #[9]
    t.start()
    s.sniff()

This last bit of code should be fairly straightforward to understand. We define a simple string signature __[1]__ so that we can test that the responses are coming from UDP packets that we sent originally. Our __udp_sender__ function __[2]__ simply takes in a subnet that we specify at the top of our script, iterates through all IP addresses in that subnet, and fires UDP datagram at them.

We then define a __Scanner__ class __[3]__. To initialize it, we pass it a host as an argument. As it initializes, we create a socket, turn on promiscuous mode if running Winzozz, and make the socket an attribute of the __Scanner__ class.

The __sniff__ method __[4]__ sniffs the network, following the same steps as in the previous example, except that this time it keeps a record of which hosts are up. If we detect the anticipated ICMP message, we first check to make sure that the ICMP response is coming from within our target subnet __[5]__. We then perform our final check of making sure that the ICMP response has our magic string in it __[6]__. If all of these checks pass, we print out the IP address of the host where the ICMP message originated __[7]__. When we end the sniffing process by using CTRL-C, we handle the keyboard interrupt __[8]__. That is, we turn off promiscuous mode if on Winzozz and print out a sorted list of live hosts.

The __\_\_main\_\___ block does the work of setting things up: It creates the __Scanner__ object, sleeps just a few seconds, and then, before calling the __sniff__ method, spawns __udp_sender__ in a separate thread __[9]__ to ensure that we aren't interfering with our ability to sniff responses. Let's try it out.

### *__The ipaddress module__*

Our scanner will use a library called __ipaddress__, which will allow us to feed in a subnet mask such as 192.168.0.0/24 and have our scanner handle it appropriately.

The __ipaddress__ module makes working with subnets and addressing very easy. For example, you can run simple tests like the following using the __Ipv4Network__ object:

In [ ]:
ip_address = "192.168.112.3"

if ip_address in Ipv4Network("192.168.112.0/24"):
    print(True)

Or we can create simple iterators if you want to send packets to an entire network:

In [ ]:
for ip in Ipv4Network("192.168.112.1/24"):
    s = socket.socket()
    s.connect((ip, 25))
    # Send mail packets

This will greatly simplify your programming life when dealing with entire networks at a time, and it is ideally suited for our host discovery tool.

# *__Kicking the Tires__*

Now let's take our scanner and run it against the local network. You can use Linux or Winzozz for this, as the results will be the same. In the authors' case, the IP address of the local machine we were on was 192.168.0.187, so we set our scanner to hit 192.168.0.0/24. If the output is too noisy when you run your scanner, simply comment out all print statements except for the last one that tells you what hosts are responding.
```
python.exe scanner.py
Host Up: 192.168.0.1
Host Up: 192.168.0.190
Host Up: 192.168.0.192
Host Up: 192.168.0.195
```
For a quick scan like the one we performed, it took only a few seconds to get the results. By cross-referencing these IP addresses with the DHCP table in a home router, we were able to verify that the results were accurate.
You can easily expand what you've learned in this chapter to decode TCP and UDP packets as well as to build additional tooling around the scanner.
This scanner is also useful for the trojan framework we will begin building in Chapter 7. This would allow a deployed trojan to scan the local network for additional targets.
Now that you know the basics of how networks work on a high and low level, let's explore a very mature Python library called __Scapy__.